Get Common Genes

The following code is to get the common variable genes between the data, and create a file intersections_genes.pkl for later analysis with BuDDi.  

In [23]:
# general imports
import warnings
import numpy as np
import os
import pandas as pd
import scipy as sp
from scipy.sparse import coo_matrix
import collections
import scanpy as sc


# Images, plots, display, and visualization
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.manifold import TSNE
import sklearn as sk

# matplotlib settings for Jupyter notebooks only
%matplotlib inline

import pickle
import gzip
from pathlib import Path


In [24]:
# pathds for data
os.chdir("/beevol/home/ivicha/Rotation_project/")
aug_data_path = f"{os.getcwd()}/data/single_cell_data/augmented_cortex_data/"
cybersort_path = f"{os.getcwd()}/data/single_cell_data/cybersort_cortex/"
data_path = f"{os.getcwd()}/data/single_cell_data/cortex6k/matrix.csv"
scpred_path = f"{os.getcwd()}/data/single_cell_data/cortexcell_labels/metadata.csv"

In [25]:
#checking pseudobulk creating
res_name = "cortex6k"
check_name = f"{res_name}_pseudo_4"
check_path = f"{aug_data_path}/{check_name}.pkl"
check_df = pickle.load( open( check_path, "rb" ) )
check_df

gene_ids,3.8-1.2,3.8-1.3,3.8-1.4,3.8-1.5,5-HT3C2,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,...,ZXDB,ZXDC,ZYG11A,ZYG11AP1,ZYG11B,ZYX,ZZEF1,ZZZ3,bA255A11.4,bA395L14.12
0,0,11288,0,0,2328,8251,240,18888,1048531,56700,...,6607,307363,4132,0,523646,34158,157209,471510,12,1392
0,50,1879,64,63,3143,39675,3711,13776,2462848,23156,...,11734,253852,3224,0,1031467,50531,189045,493316,144,5796
0,0,56,42,380,3706,28809,2164,9877,7834132,22495,...,18766,254194,1813,0,958677,36562,196930,689746,7,3601
0,21,2316,188,162,2674,16783,2958,15232,4039164,27388,...,6017,177022,4012,0,919677,42099,134179,283548,84,2590
0,120,130,192,261,4007,22479,3824,13816,577632,29934,...,3822,194422,3496,0,759412,30907,100301,347009,181,3738
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,3,2009,86,321,2407,24190,1476,5568,7613439,17616,...,14940,273368,2515,0,757857,47239,152228,602630,106,1929
0,76,10,174,14,212,13787,2298,13002,9887472,12239,...,19836,226460,2719,0,1127624,86822,184128,310301,14,6017
0,78,4491,102,39,2060,33951,2634,18071,981437,36027,...,5995,225957,3450,0,877467,57962,143204,469086,329,1690
0,1,524,0,132,6420,113620,3506,7766,1148741,34548,...,8619,172380,1486,0,546318,23276,162565,541170,73,2006


In [26]:
# 6k process below
# read in the data
adata = sc.read_csv(data_path)
adata.var_names_make_unique()  # this is unnecessary if using `var_names='gene_ids'` in `sc.read_10x_mtx`
adata.var['gene_ids'] = adata.var_names

In [27]:

# add metadata
meta_data = pd.read_csv(f"{scpred_path}", sep="\,")
#if barcodes are available, use the following:
#barcodes = pd.read_csv(f"{data_path}/barcodes.tsv", header=None, names=['code']) 
#meta_df = barcodes.join(other=meta_data, on=['code'], how='left', sort=False)

#file contains many cell types, here we're simplifying into 6 types only:
#oligos
meta_data = meta_data.replace(to_replace=['Oligo L4-6 OPALIN', 'Oligo L4-6 MOBP COL18A1'],value=['Oligodendrocyte', 'Oligodendrocyte'])
#microglia
meta_data = meta_data.replace(to_replace=['Micro L1-6 C1QC'], value=['Microglia'])
#Astrocyes
meta_data = meta_data.replace(to_replace=['Astro L1-6 FGFR3 ETNPPL', 'Astro L1 FGFR3 MT1G','Astro L1 FGFR3 FOS'], value=['Astrocyte', 'Astrocyte', 'Astrocyte'])
#opcs
meta_data = meta_data.replace(to_replace=['OPC L1-6 MYT1'], value= ['OPC'])
#endocytes
meta_data = meta_data.replace(to_replace=['Endo L2-5 CLDN5'], value=['Endocyte'])
#others
meta_data = meta_data.fillna('')
meta_data = meta_data.replace(to_replace=['VLMC L1-3 CYP1B1', 'Peri L1-6 MUSTN1', 'NA', ''], value= ['Other', 'Other', 'Other', 'Other'])
#neurons
meta_data = meta_data.replace(regex=r'Inh', value= 'Neuron')
meta_data = meta_data.replace(regex=r'Exc', value = 'Neuron')

meta_data = meta_data.replace(regex='Neuron\s\w\w\s\w*\w*', value = 'Neuron')
meta_data = meta_data.replace(regex='Neuron\s\w\w\W\s\w*\w*', value = 'Neuron')

meta_data = meta_data.replace(regex='Neuron\s\w*', value = 'Neuron')

meta_data = meta_data.replace(regex='Neuron\W\w\s\w*', value = 'Neuron')

meta_data = meta_data.replace(regex='Neuron\s\w*', value = 'Neuron')

/beevol/home/ivicha/miniconda3/envs/sc_bulk_ood/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [28]:
#assigning into variable to match cell predictors, and make into category type.
meta_data['scpred_prediction']= meta_data.cell_type_alias_label
meta_data.cell_type_alias_label = meta_data.cell_type_alias_label.astype('category')

#assigning celltypes from meta_data into adata
adata.obs['scpred_CellType'] = meta_data.cell_type_alias_label.values
adata.obs['CellType'] = meta_data.cell_type_alias_label.values

# filter out cells with less than 200 genes and genes expressed in less than 3 cells
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)

# remove genes with high mitochondrial content
adata.var['mt'] = adata.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

# slice the data based on the plots from above
# remove cells with more than 2000 genes
# remove cells with more than 10% MTgenes
adata = adata[adata.obs.n_genes_by_counts < 2000, :]
adata = adata[adata.obs.pct_counts_mt < 10, :]

# normalize to 10K counts per cell
sc.pp.normalize_total(adata, target_sum=1e4)

# log data
sc.pp.log1p(adata)

# get high variance genes
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
cortex_6k_gene_ids = adata.var['gene_ids']

# and filter
adata = adata[:, adata.var.highly_variable]

/beevol/home/ivicha/miniconda3/envs/sc_bulk_ood/lib/python3.9/site-packages/scanpy/preprocessing/_normalization.py:170: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


In [29]:
cortex_6k_gene_ids_variable = adata.var['gene_ids']

In [30]:
genes_all = adata.var['gene_ids']

genes_all = list(set.intersection(*[set(x) for x in genes_all]))
len(genes_all)

0

In [31]:
genes_all_variable = adata.var['gene_ids']

genes_all_variable = list(set.intersection(*[set(x) for x in genes_all_variable]))
len(genes_all_variable)

0

In [32]:
# write out the gene ids
gene_out_file = os.path.join(aug_data_path, "intersection_genes.pkl")
gene_out_path = Path(gene_out_file)
pickle.dump( genes_all, open(gene_out_path, "wb" ) )